In [1]:
from collections import defaultdict
import geopandas as gp
from scipy.stats import wilcoxon, pearsonr, spearmanr, kendalltau
from collections import defaultdict
import pandas as pd
from src.constants import BniaIndicators
from src.exp_helper import *
from sklearn.metrics import r2_score, explained_variance_score
from IPython.display import display

In [2]:
POP = pd.read_excel('data/open-baltimore/raw/VS16_Indicators_2010-2016.xlsx',sheet_name='2010')
POP = POP.set_index('CSA2010').drop('Baltimore City')
POP = POP['Total Population']
IDCTR = pd.read_excel('data/open-baltimore/raw/VS16_Indicators_2010-2016.xlsx',sheet_name='2015')
IDCTR = IDCTR.set_index('CSA2010').drop('Baltimore City')

In [18]:
INCOME_COLS = [col for col in BniaIndicators.household_income if col in set(IDCTR.columns.tolist())]
HOUSING_COLS = [col for col in BniaIndicators.housing if col in set(IDCTR.columns.tolist())]

In [19]:
TEST_CHOICES = [(pearsonr,'pearsonr'), (spearmanr,'spearmanr'), (kendalltau,'kendalltau')]    

In [20]:
SIG_LVL=0.05

# get pred_rest

In [7]:
def iter_keys():
    for xday in [2, 7]:
        for fpn in ['fp','fn', 'tp']:
            for hname in ['top20', 'above_mean', 'above_mean_std']:
                for normed in [True, False]:
                    yield xday, fpn, hname, normed
def keys2name(xday, fpn, hname, normed):
    s = '%dd-%s-%s' % (xday, fpn, hname)
    if normed:
        return s+'-normed'
    return s + '-raw'

In [8]:
def norm_by_population(df):
    for p in df.index:
#         print(p)
        for c in ['bower','kde200']:
            arr = df.loc[p,c]
            df.loc[p,c]=arr/(POP.values)


In [9]:
# pred_res[xday][fpn][hname][normed]
pred_res = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
for xday, fpn, hname, normed in iter_keys():
    df = pd.read_csv('exp_res/bower_%dday_bnia_%s_hotspots_%s.csv' % (xday, hname, fpn), index_col=0).applymap(eval)
    if normed: 
        norm_by_population(df)
    pred_res[xday][fpn][hname][normed]=df

# stest on stats across all periods

In [10]:
def statistical_test_stat_periods(pred_df, test, idctr_cols):
    stest = []
    # get stats of both methods across periods
    stat_bower = pd.DataFrame(pred_df.bower.tolist()).describe()
    stat_kde200 = pd.DataFrame(pred_df.kde200.tolist()).describe()
#     for stat_name in ['mean','min','max','50%']:
    for stat_name in ['mean',]:
        b_stat = stat_bower.loc[stat_name]
        k_stat = stat_kde200.loc[stat_name]    
        for icol in idctr_cols:
            res = {'stat_name': stat_name, 'indicator': icol}
            idctr = IDCTR[icol].values
            # bower
            r_bower, p_bower = test(b_stat, idctr)
            res['r_bower'] = r_bower
            res['p_bower'] = p_bower
            res['rp_bower'] = 'r=%0.4f, p=%0.4f' % (r_bower, p_bower)
            # kde 
            r_kde200, p_kde200 = test(k_stat, idctr)
            res['r_kde200'] = r_kde200
            res['p_kde200'] = p_kde200
            res['rp_kde200'] = 'r=%0.4f, p=%0.4f' % (r_kde200, p_kde200)
            stest.append(res)
    return pd.DataFrame(stest)

In [12]:
%%time
# compute stest result for each combination of xday, fpn, hname, normed
all_stest_stat = {}
for test, tname in TEST_CHOICES:
    stat_idctr_stest_rp = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
    for xday, fpn, hname, normed in iter_keys():
        df = pred_res[xday][fpn][hname][normed]
        stat_idctr_stest_rp[xday][fpn][hname][normed]= statistical_test_stat_periods(df, test, income_cols)
    all_stest_stat[tname]=stat_idctr_stest_rp

CPU times: user 27.3 s, sys: 52 ms, total: 27.3 s
Wall time: 27.3 s


In [21]:
%%time
idctr_cols = INCOME_COLS

all_stest_stat_report_table = {}
for _, tname in TEST_CHOICES:
    stat_idctr_stest_rp = all_stest_stat[tname]
    m_stat_sig = []
    for xday, fpn, hname, normed in iter_keys():
        stest = stat_idctr_stest_rp[xday][fpn][hname][normed]
        for mname in ['bower', 'kde200']:
            for stat_name in ['mean',]:
                m_stat_rp = stest[stest.stat_name==stat_name].set_index('indicator').drop('stat_name', axis=1)
                m_stat_rp = m_stat_rp[m_stat_rp[f"p_{mname}"]<SIG_LVL][f'rp_{mname}']
                m_stat_rp.name = '%s-%s-%s' % (mname, stat_name, keys2name(xday, fpn, hname, normed))
                m_stat_sig.append(m_stat_rp)
            
    report_table =pd.DataFrame(m_stat_sig).T.reindex(idctr_cols).fillna('')
    report_table.index = [col.replace('$','\$') for col in report_table.index]
    tuples = [col.split('-') for col in report_table.columns]
    index = pd.MultiIndex.from_tuples(tuples)
    report_table.columns = index
    all_stest_stat_report_table[tname] = report_table


CPU times: user 632 ms, sys: 4 ms, total: 636 ms
Wall time: 620 ms


In [22]:
tname = 'kendalltau'  # pearsonr spearmanr kendalltau
report_table = all_stest_stat_report_table[tname]
for mname in ('bower','kde200'):
    for stat_name in ['mean',]:
        for xday in (2,7):
            xday_name=f'{xday}d'
            print()
            tmp_df = report_table.loc[:, (mname, stat_name, xday_name)].copy()
            tmp_df.index.name = '%s-%s-%s-%s' % (tname, mname, stat_name, xday_name)
            display(tmp_df)
            print('-'*10)
            print('-'*10)


/home/Jiahui/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


fp  \
                                                                 top20   
                                                                normed   
kendalltau-bower-mean-2d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000    r=0.1825, p=0.0491   
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.2822, p=0.0024   
Percent of Children Living Below the Poverty Line   r=0.3037, p=0.0011   

                                                                        \
                                                                         
                                                                   raw   
kendalltau-bower-mean-2d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.1973, p=0.0334   
Percent of Children Living Below the Poverty Line   r=0.2135, p=0.0214   

                                                                         \
                                                             above_mean   
                                                                 normed   
kendalltau-bower-mean-2d                                                  
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000                        
Percent of Households Earning More than \$75,000    r=-0.1946, p=0.0359   
Percent of Family Households Living Below the P...   r=0.2458, p=0.0081   
Percent of Children Living Below the Poverty Line    r=0.2458, p=0.0081   

                                                                        \
                                                                         
                                                                   raw   
kendalltau-bower-mean-2d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.1852, p=0.0459   
Percent of Children Living Below the Poverty Line   r=0.1960, p=0.0346   

                                                                        \
                                                        above_mean_std   
                                                                normed   
kendalltau-bower-mean-2d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000      

----------
----------



fp  \
                                                                 top20   
                                                                normed   
kendalltau-bower-mean-7d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.2781, p=0.0027   
Percent of Children Living Below the Poverty Line   r=0.3024, p=0.0011   

                                                                        \
                                                                         
                                                                   raw   
kendalltau-bower-mean-7d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.1987, p=0.0322   
Percent of Children Living Below the Poverty Line   r=0.2148, p=0.0206   

                                                                         \
                                                             above_mean   
                                                                 normed   
kendalltau-bower-mean-7d                                                  
Median Household Income                                                   
Percent of Households Earning Less than \$25,000     r=0.1825, p=0.0491   
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000                        
Percent of Households Earning More than \$75,000    r=-0.1906, p=0.0399   
Percent of Family Households Living Below the P...   r=0.2444, p=0.0084   
Percent of Children Living Below the Poverty Line    r=0.2444, p=0.0084   

                                                                        \
                                                                         
                                                                   raw   
kendalltau-bower-mean-7d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.1838, p=0.0475   
Percent of Children Living Below the Poverty Line   r=0.1946, p=0.0359   

                                                                        \
                                                        above_mean_std   
                                                                normed   
kendalltau-bower-mean-7d                                                 
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000      

----------
----------



fp  \
                                                                  top20   
                                                                 normed   
kendalltau-kde200-mean-2d                                                 
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000  r=-0.1838, p=0.0475   
Percent of Households Earning More than \$75,000                          
Percent of Family Households Living Below the P...   r=0.2620, p=0.0047   
Percent of Children Living Below the Poverty Line    r=0.2889, p=0.0018   

                                                                        \
                                                                         
                                                                   raw   
kendalltau-kde200-mean-2d                                                
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.2310, p=0.0128   
Percent of Children Living Below the Poverty Line   r=0.2525, p=0.0065   

                                                                         \
                                                             above_mean   
                                                                 normed   
kendalltau-kde200-mean-2d                                                 
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000                        
Percent of Households Earning More than \$75,000    r=-0.1865, p=0.0443   
Percent of Family Households Living Below the P...   r=0.2512, p=0.0068   
Percent of Children Living Below the Poverty Line    r=0.2593, p=0.0052   

                                                                         \
                                                                          
                                                                    raw   
kendalltau-kde200-mean-2d                                                 
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000                        
Percent of Households Earning More than \$75,000    r=-0.1906, p=0.0399   
Percent of Family Households Living Below the P...   r=0.2040, p=0.0278   
Percent of Children Living Below the Poverty Line    r=0.2256, p=0.0150   

                                                                        \
                                                        above_mean_std   
                                                                normed   
kendalltau-kde200-mean-2d                                                
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$2

----------
----------



fp  \
                                                                  top20   
                                                                 normed   
kendalltau-kde200-mean-7d                                                 
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000  r=-0.1825, p=0.0491   
Percent of Households Earning More than \$75,000                          
Percent of Family Households Living Below the P...   r=0.2579, p=0.0054   
Percent of Children Living Below the Poverty Line    r=0.2875, p=0.0019   

                                                                        \
                                                                         
                                                                   raw   
kendalltau-kde200-mean-7d                                                
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$25,000 to \$40,000                       
Percent of Households Earning \$40,000 to \$60,000                       
Percent of Households Earning \$60,000 to \$75,000                       
Percent of Households Earning More than \$75,000                         
Percent of Family Households Living Below the P...  r=0.2283, p=0.0139   
Percent of Children Living Below the Poverty Line   r=0.2525, p=0.0065   

                                                                         \
                                                             above_mean   
                                                                 normed   
kendalltau-kde200-mean-7d                                                 
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000                        
Percent of Households Earning More than \$75,000    r=-0.1879, p=0.0428   
Percent of Family Households Living Below the P...   r=0.2471, p=0.0077   
Percent of Children Living Below the Poverty Line    r=0.2579, p=0.0054   

                                                                         \
                                                                          
                                                                    raw   
kendalltau-kde200-mean-7d                                                 
Median Household Income                                                   
Percent of Households Earning Less than \$25,000                          
Percent of Households Earning \$25,000 to \$40,000                        
Percent of Households Earning \$40,000 to \$60,000                        
Percent of Households Earning \$60,000 to \$75,000                        
Percent of Households Earning More than \$75,000    r=-0.1865, p=0.0443   
Percent of Family Households Living Below the P...   r=0.2000, p=0.0311   
Percent of Children Living Below the Poverty Line    r=0.2242, p=0.0156   

                                                                        \
                                                        above_mean_std   
                                                                normed   
kendalltau-kde200-mean-7d                                                
Median Household Income                                                  
Percent of Households Earning Less than \$25,000                         
Percent of Households Earning \$2

----------
----------


# test per period

In [18]:
def statistical_test_per_period(pred_df, test, idctr_cols):
    stest = []
    for period, (bower, kde200) in pred_df.iterrows():
        for icol in idctr_cols:
            res = {'period': period, 'indicator': icol}
            idctr = IDCTR[icol].values
            # bower
            r_bower, p_bower = test(bower, idctr)
            res['r_bower'] = r_bower
            res['p_bower'] = p_bower
            res['rp_bower'] = 'r=%0.4f, p=%0.4f' % (r_bower, p_bower)
            # kde 
            r_kde200, p_kde200 = test(kde200, idctr)
            res['r_kde200'] = r_kde200
            res['p_kde200'] = p_kde200
            res['rp_kde200'] = 'r=%0.4f, p=%0.4f' % (r_kde200, p_kde200)
            stest.append(res)
    
    return pd.DataFrame(stest)

In [19]:
def keep_sig_per_period(df, sig_lvl, mname, periods):
    pivoted = df[df['p_'+mname]<sig_lvl].pivot(index='period', columns='indicator', values='rp_'+mname) 
    return pivoted.reindex(periods)
def sig_pcnt_periods(df, sig_lvl, mname, periods):
    ires = keep_sig_per_period(df, sig_lvl, mname, periods)
    return (~ires.isnull()).mean()

In [20]:
all_periods = {2: pred_res[2]['fn']['above_mean_std'][False].index, 7: pred_res[7]['fn']['above_mean_std'][False].index} # key: xday

In [74]:
sig_lvl = 0.05
sum_table_each_test = {}
for test, tname in test_choices:
    period_idctr_stest_rp = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
    for xday, fpn, hname, normed in iter_keys():
        df = pred_res[xday][fpn][hname][normed]
        period_idctr_stest_rp[xday][fpn][hname][normed]= statistical_test_per_period(df, test, income_cols)
    sig_pcnt_of_each_method = []
    for mname in ['bower', 'kde200']:
        for xday, fpn, hname, normed in iter_keys():
            stest = period_idctr_stest_rp[xday][fpn][hname][normed]
            sig_pcnt = sig_pcnt_periods(stest, sig_lvl, mname, all_periods[xday])
            sig_pcnt.name = '%s-%s' % (mname, keys2name(xday, fpn, hname, normed))
            sig_pcnt_of_each_method.append(sig_pcnt)
    sum_table =pd.DataFrame(sig_pcnt_of_each_method).T.reindex(income_cols).fillna(0)
    sum_table.index = [col.replace('$','\$') for col in sum_table.index]
    tuples = [col.split('-') for col in sum_table.columns]
    index = pd.MultiIndex.from_tuples(tuples)
    sum_table.columns = index
    sum_table_each_test[tname] = sum_table

In [75]:
tname = 'kendalltau'  # pearsonr spearmanr kendalltau
sum_table = sum_table_each_test[tname]
for mname in ('bower','kde200'):
    for xday in ('2d','7d'):
        print()
        tmp_df = sum_table.loc[:, (mname, xday)].copy()
        tmp_df.index.name = '%s-%s-%s' % (tname, mname, xday)
        display(tmp_df)
        print('    ')
        print('---------------------')

fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-2d                                                      
Median Household Income                             0.069536  0.049669   
Percent of Households Earning Less than \$25,000    0.195364  0.079470   
Percent of Households Earning \$25,000 to \$40,000  0.009934  0.052980   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.109272  0.006623   
Percent of Households Earning More than \$75,000    0.145695  0.178808   
Percent of Family Households Living Below the P...  0.516556  0.417219   
Percent of Children Living Below the Poverty Line   0.582781  0.460265   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-2d                                                       
Median Household Income                              0.145695  0.069536   
Percent of Households Earning Less than \$25,000     0.264901  0.039735   
Percent of Households Earning \$25,000 to \$40,000   0.003311  0.072848   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006623   
Percent of Households Earning \$60,000 to \$75,000   0.069536  0.000000   
Percent of Households Earning More than \$75,000     0.331126  0.357616   
Percent of Family Households Living Below the P...   0.552980  0.417219   
Percent of Children Living Below the Poverty Line    0.658940  0.486755   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-bower-2d                                                           
Median Household Income                                  0.026490  0.019868   
Percent of Households Earning Less than \$25,000         0.079470  0.039735   
Percent of Households Earning \$25,000 to \$40,000       0.029801  0.019868   
Percent of Households Earning \$40,000 to \$60,000       0.006623  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.036424  0.009934   
Percent of Households Earning More than \$75,000         0.023179  0.033113   
Percent of Family Households Living Below the P...       0.314570  0.301325   
Percent of Children Living Below the Poverty Line        0.354305  0.258278   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-2d                                                      
Median Household Income                             0.046358  0.046358   
Percent of Households Earning Less than \$25,000    0.056291  0.059603   
Percent of Households Earning \$25,000 to \$40,000  0.049669  0.043046   
Percent of Households Earning \$40,000 to \$60,000  0.046358  0.049669   
Percent of Households Earning \$60,000 to \$75,000  0.029801  0.026490   
Percent of Households Earning More than \$75,000    0.046358  0.029801   
Percent of Family Households Living Below the P...  0.076159  0.062914   
Percent of Children Living Below the Poverty Line   0.082781  0.072848   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-2d                                                       
Median Household Income                              0.049669  0.039735   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-7d                                                      
Median Household Income                             0.074074  0.067340   
Percent of Households Earning Less than \$25,000    0.198653  0.090909   
Percent of Households Earning \$25,000 to \$40,000  0.010101  0.057239   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.117845  0.006734   
Percent of Households Earning More than \$75,000    0.154882  0.191919   
Percent of Family Households Living Below the P...  0.518519  0.434343   
Percent of Children Living Below the Poverty Line   0.585859  0.464646   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-7d                                                       
Median Household Income                              0.121212  0.084175   
Percent of Households Earning Less than \$25,000     0.252525  0.040404   
Percent of Households Earning \$25,000 to \$40,000   0.003367  0.074074   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006734   
Percent of Households Earning \$60,000 to \$75,000   0.077441  0.000000   
Percent of Households Earning More than \$75,000     0.319865  0.350168   
Percent of Family Households Living Below the P...   0.542088  0.417508   
Percent of Children Living Below the Poverty Line    0.653199  0.491582   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-bower-7d                                                           
Median Household Income                                  0.023569  0.020202   
Percent of Households Earning Less than \$25,000         0.077441  0.043771   
Percent of Households Earning \$25,000 to \$40,000       0.030303  0.020202   
Percent of Households Earning \$40,000 to \$60,000       0.006734  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.037037  0.010101   
Percent of Households Earning More than \$75,000         0.023569  0.033670   
Percent of Family Households Living Below the P...       0.319865  0.306397   
Percent of Children Living Below the Poverty Line        0.373737  0.262626   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-7d                                                      
Median Household Income                             0.043771  0.030303   
Percent of Households Earning Less than \$25,000    0.033670  0.023569   
Percent of Households Earning \$25,000 to \$40,000  0.047138  0.090909   
Percent of Households Earning \$40,000 to \$60,000  0.026936  0.040404   
Percent of Households Earning \$60,000 to \$75,000  0.006734  0.006734   
Percent of Households Earning More than \$75,000    0.053872  0.050505   
Percent of Family Households Living Below the P...  0.043771  0.030303   
Percent of Children Living Below the Poverty Line   0.050505  0.033670   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-7d                                                       
Median Household Income                              0.057239  0.047138   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-2d                                                     
Median Household Income                             0.036424  0.049669   
Percent of Households Earning Less than \$25,000    0.149007  0.109272   
Percent of Households Earning \$25,000 to \$40,000  0.000000  0.046358   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.251656  0.000000   
Percent of Households Earning More than \$75,000    0.079470  0.231788   
Percent of Family Households Living Below the P...  0.579470  0.490066   
Percent of Children Living Below the Poverty Line   0.695364  0.543046   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-2d                                                      
Median Household Income                              0.142384  0.149007   
Percent of Households Earning Less than \$25,000     0.254967  0.086093   
Percent of Households Earning \$25,000 to \$40,000   0.000000  0.052980   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.016556   
Percent of Households Earning \$60,000 to \$75,000   0.052980  0.000000   
Percent of Households Earning More than \$75,000     0.307947  0.324503   
Percent of Family Households Living Below the P...   0.609272  0.509934   
Percent of Children Living Below the Poverty Line    0.758278  0.526490   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-kde200-2d                                                          
Median Household Income                                  0.019868  0.009934   
Percent of Households Earning Less than \$25,000         0.165563  0.119205   
Percent of Households Earning \$25,000 to \$40,000       0.000000  0.013245   
Percent of Households Earning \$40,000 to \$60,000       0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.105960  0.003311   
Percent of Households Earning More than \$75,000         0.056291  0.066225   
Percent of Family Households Living Below the P...       0.367550  0.370861   
Percent of Children Living Below the Poverty Line        0.410596  0.374172   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-2d                                                     
Median Household Income                             0.036424  0.033113   
Percent of Households Earning Less than \$25,000    0.036424  0.033113   
Percent of Households Earning \$25,000 to \$40,000  0.043046  0.066225   
Percent of Households Earning \$40,000 to \$60,000  0.052980  0.059603   
Percent of Households Earning \$60,000 to \$75,000  0.029801  0.033113   
Percent of Households Earning More than \$75,000    0.043046  0.036424   
Percent of Family Households Living Below the P...  0.076159  0.059603   
Percent of Children Living Below the Poverty Line   0.066225  0.046358   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-2d                                                      
Median Household Income                              0.026490  0.026490   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-7d                                                     
Median Household Income                             0.037037  0.047138   
Percent of Households Earning Less than \$25,000    0.154882  0.114478   
Percent of Households Earning \$25,000 to \$40,000  0.000000  0.047138   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.235690  0.000000   
Percent of Households Earning More than \$75,000    0.080808  0.249158   
Percent of Family Households Living Below the P...  0.589226  0.501684   
Percent of Children Living Below the Poverty Line   0.713805  0.552189   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-7d                                                      
Median Household Income                              0.148148  0.151515   
Percent of Households Earning Less than \$25,000     0.252525  0.094276   
Percent of Households Earning \$25,000 to \$40,000   0.000000  0.053872   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.013468   
Percent of Households Earning \$60,000 to \$75,000   0.053872  0.000000   
Percent of Households Earning More than \$75,000     0.303030  0.313131   
Percent of Family Households Living Below the P...   0.599327  0.501684   
Percent of Children Living Below the Poverty Line    0.754209  0.531987   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-kde200-7d                                                          
Median Household Income                                  0.023569  0.006734   
Percent of Households Earning Less than \$25,000         0.164983  0.117845   
Percent of Households Earning \$25,000 to \$40,000       0.000000  0.013468   
Percent of Households Earning \$40,000 to \$60,000       0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.107744  0.000000   
Percent of Households Earning More than \$75,000         0.053872  0.067340   
Percent of Family Households Living Below the P...       0.373737  0.377104   
Percent of Children Living Below the Poverty Line        0.420875  0.380471   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-7d                                                     
Median Household Income                             0.040404  0.023569   
Percent of Households Earning Less than \$25,000    0.033670  0.016835   
Percent of Households Earning \$25,000 to \$40,000  0.047138  0.067340   
Percent of Households Earning \$40,000 to \$60,000  0.033670  0.060606   
Percent of Households Earning \$60,000 to \$75,000  0.006734  0.010101   
Percent of Households Earning More than \$75,000    0.043771  0.040404   
Percent of Family Households Living Below the P...  0.040404  0.016835   
Percent of Children Living Below the Poverty Line   0.047138  0.043771   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-7d                                                      
Median Household Income                              0.033670  0.020202   
Percent of Households Earning Less than \$25,000     0.

    
---------------------


In [23]:
tname = 'kendalltau'  # pearsonr spearmanr kendalltau
sum_table = sum_table_each_test[tname]
for mname in ('bower','kde200'):
    for xday in ('2d','7d'):
        print()
        tmp_df = sum_table.loc[:, (mname, xday)].copy()
        tmp_df.index.name = '%s-%s-%s' % (tname, mname, xday)
        display(tmp_df)
        print('    ')
        print('---------------------')

fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-2d                                                      
Median Household Income                             0.069536  0.049669   
Percent of Households Earning Less than \$25,000    0.195364  0.079470   
Percent of Households Earning \$25,000 to \$40,000  0.009934  0.052980   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.109272  0.006623   
Percent of Households Earning More than \$75,000    0.145695  0.178808   
Percent of Family Households Living Below the P...  0.516556  0.417219   
Percent of Children Living Below the Poverty Line   0.582781  0.460265   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-2d                                                       
Median Household Income                              0.145695  0.069536   
Percent of Households Earning Less than \$25,000     0.264901  0.039735   
Percent of Households Earning \$25,000 to \$40,000   0.003311  0.072848   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006623   
Percent of Households Earning \$60,000 to \$75,000   0.069536  0.000000   
Percent of Households Earning More than \$75,000     0.331126  0.357616   
Percent of Family Households Living Below the P...   0.552980  0.417219   
Percent of Children Living Below the Poverty Line    0.658940  0.486755   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-bower-2d                                                           
Median Household Income                                  0.026490  0.019868   
Percent of Households Earning Less than \$25,000         0.079470  0.039735   
Percent of Households Earning \$25,000 to \$40,000       0.029801  0.019868   
Percent of Households Earning \$40,000 to \$60,000       0.006623  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.036424  0.009934   
Percent of Households Earning More than \$75,000         0.023179  0.033113   
Percent of Family Households Living Below the P...       0.314570  0.301325   
Percent of Children Living Below the Poverty Line        0.354305  0.258278   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-2d                                                      
Median Household Income                             0.046358  0.046358   
Percent of Households Earning Less than \$25,000    0.056291  0.059603   
Percent of Households Earning \$25,000 to \$40,000  0.049669  0.043046   
Percent of Households Earning \$40,000 to \$60,000  0.046358  0.049669   
Percent of Households Earning \$60,000 to \$75,000  0.029801  0.026490   
Percent of Households Earning More than \$75,000    0.046358  0.029801   
Percent of Family Households Living Below the P...  0.076159  0.062914   
Percent of Children Living Below the Poverty Line   0.082781  0.072848   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-2d                                                       
Median Household Income                              0.049669  0.039735   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-7d                                                      
Median Household Income                             0.074074  0.067340   
Percent of Households Earning Less than \$25,000    0.198653  0.090909   
Percent of Households Earning \$25,000 to \$40,000  0.010101  0.057239   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.117845  0.006734   
Percent of Households Earning More than \$75,000    0.154882  0.191919   
Percent of Family Households Living Below the P...  0.518519  0.434343   
Percent of Children Living Below the Poverty Line   0.585859  0.464646   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-7d                                                       
Median Household Income                              0.121212  0.084175   
Percent of Households Earning Less than \$25,000     0.252525  0.040404   
Percent of Households Earning \$25,000 to \$40,000   0.003367  0.074074   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006734   
Percent of Households Earning \$60,000 to \$75,000   0.077441  0.000000   
Percent of Households Earning More than \$75,000     0.319865  0.350168   
Percent of Family Households Living Below the P...   0.542088  0.417508   
Percent of Children Living Below the Poverty Line    0.653199  0.491582   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-bower-7d                                                           
Median Household Income                                  0.023569  0.020202   
Percent of Households Earning Less than \$25,000         0.077441  0.043771   
Percent of Households Earning \$25,000 to \$40,000       0.030303  0.020202   
Percent of Households Earning \$40,000 to \$60,000       0.006734  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.037037  0.010101   
Percent of Households Earning More than \$75,000         0.023569  0.033670   
Percent of Family Households Living Below the P...       0.319865  0.306397   
Percent of Children Living Below the Poverty Line        0.373737  0.262626   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-7d                                                      
Median Household Income                             0.043771  0.030303   
Percent of Households Earning Less than \$25,000    0.033670  0.023569   
Percent of Households Earning \$25,000 to \$40,000  0.047138  0.090909   
Percent of Households Earning \$40,000 to \$60,000  0.026936  0.040404   
Percent of Households Earning \$60,000 to \$75,000  0.006734  0.006734   
Percent of Households Earning More than \$75,000    0.053872  0.050505   
Percent of Family Households Living Below the P...  0.043771  0.030303   
Percent of Children Living Below the Poverty Line   0.050505  0.033670   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-7d                                                       
Median Household Income                              0.057239  0.047138   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-2d                                                     
Median Household Income                             0.069536  0.049669   
Percent of Households Earning Less than \$25,000    0.195364  0.079470   
Percent of Households Earning \$25,000 to \$40,000  0.009934  0.052980   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.109272  0.006623   
Percent of Households Earning More than \$75,000    0.145695  0.178808   
Percent of Family Households Living Below the P...  0.516556  0.417219   
Percent of Children Living Below the Poverty Line   0.582781  0.460265   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-2d                                                      
Median Household Income                              0.145695  0.069536   
Percent of Households Earning Less than \$25,000     0.264901  0.039735   
Percent of Households Earning \$25,000 to \$40,000   0.003311  0.072848   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006623   
Percent of Households Earning \$60,000 to \$75,000   0.069536  0.000000   
Percent of Households Earning More than \$75,000     0.331126  0.357616   
Percent of Family Households Living Below the P...   0.552980  0.417219   
Percent of Children Living Below the Poverty Line    0.658940  0.486755   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-kde200-2d                                                          
Median Household Income                                  0.026490  0.019868   
Percent of Households Earning Less than \$25,000         0.079470  0.039735   
Percent of Households Earning \$25,000 to \$40,000       0.029801  0.019868   
Percent of Households Earning \$40,000 to \$60,000       0.006623  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.036424  0.009934   
Percent of Households Earning More than \$75,000         0.023179  0.033113   
Percent of Family Households Living Below the P...       0.314570  0.301325   
Percent of Children Living Below the Poverty Line        0.354305  0.258278   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-2d                                                     
Median Household Income                             0.046358  0.046358   
Percent of Households Earning Less than \$25,000    0.056291  0.059603   
Percent of Households Earning \$25,000 to \$40,000  0.049669  0.043046   
Percent of Households Earning \$40,000 to \$60,000  0.046358  0.049669   
Percent of Households Earning \$60,000 to \$75,000  0.029801  0.026490   
Percent of Households Earning More than \$75,000    0.046358  0.029801   
Percent of Family Households Living Below the P...  0.076159  0.062914   
Percent of Children Living Below the Poverty Line   0.082781  0.072848   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-2d                                                      
Median Household Income                              0.049669  0.039735   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-7d                                                     
Median Household Income                             0.074074  0.067340   
Percent of Households Earning Less than \$25,000    0.198653  0.090909   
Percent of Households Earning \$25,000 to \$40,000  0.010101  0.057239   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.117845  0.006734   
Percent of Households Earning More than \$75,000    0.154882  0.191919   
Percent of Family Households Living Below the P...  0.518519  0.434343   
Percent of Children Living Below the Poverty Line   0.585859  0.464646   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-7d                                                      
Median Household Income                              0.121212  0.084175   
Percent of Households Earning Less than \$25,000     0.252525  0.040404   
Percent of Households Earning \$25,000 to \$40,000   0.003367  0.074074   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006734   
Percent of Households Earning \$60,000 to \$75,000   0.077441  0.000000   
Percent of Households Earning More than \$75,000     0.319865  0.350168   
Percent of Family Households Living Below the P...   0.542088  0.417508   
Percent of Children Living Below the Poverty Line    0.653199  0.491582   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-kde200-7d                                                          
Median Household Income                                  0.023569  0.020202   
Percent of Households Earning Less than \$25,000         0.077441  0.043771   
Percent of Households Earning \$25,000 to \$40,000       0.030303  0.020202   
Percent of Households Earning \$40,000 to \$60,000       0.006734  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.037037  0.010101   
Percent of Households Earning More than \$75,000         0.023569  0.033670   
Percent of Family Households Living Below the P...       0.319865  0.306397   
Percent of Children Living Below the Poverty Line        0.373737  0.262626   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-7d                                                     
Median Household Income                             0.043771  0.030303   
Percent of Households Earning Less than \$25,000    0.033670  0.023569   
Percent of Households Earning \$25,000 to \$40,000  0.047138  0.090909   
Percent of Households Earning \$40,000 to \$60,000  0.026936  0.040404   
Percent of Households Earning \$60,000 to \$75,000  0.006734  0.006734   
Percent of Households Earning More than \$75,000    0.053872  0.050505   
Percent of Family Households Living Below the P...  0.043771  0.030303   
Percent of Children Living Below the Poverty Line   0.050505  0.033670   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-7d                                                      
Median Household Income                              0.057239  0.047138   
Percent of Households Earning Less than \$25,000     0.

    
---------------------
